In [14]:
# dataset: https://www.kaggle.com/uciml/pima-indians-diabetes-database
# github: https://github.com/pranav567/AI-ML-Lab/tree/master/experiment-4

In [15]:
import pandas as pd
import numpy as np

diabetes = pd.read_csv('diabetes_dataset.csv')
diabetes = diabetes.dropna()
print(diabetes)

     Glucose  BloodPressure  SkinThickness  Insulin   BMI  Age  Outcome
0        148             72             35        0  33.6   50        1
1         85             66             29        0  26.6   31        0
2        183             64              0        0  23.3   32        1
3         89             66             23       94  28.1   21        0
4        137             40             35      168  43.1   33        1
..       ...            ...            ...      ...   ...  ...      ...
763      101             76             48      180  32.9   63        0
764      122             70             27        0  36.8   27        0
765      121             72             23      112  26.2   30        0
766      126             60              0        0  30.1   47        1
767       93             70             31        0  30.4   23        0

[768 rows x 7 columns]


In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(diabetes,diabetes['Outcome'],test_size=0.2)


In [17]:
df = x_train.copy()
diabetes_yes = df.where(df['Outcome']==1).dropna()
diabetes_yes_prob = len(diabetes_yes)/len(df)
diabetes_no = df.where(df['Outcome']==0).dropna()
diabetes_no_prob = len(diabetes_no)/len(df)

print(diabetes_yes_prob,diabetes_no_prob)

0.3583061889250814 0.6416938110749185


In [18]:
columns = df.iloc[:,:6].columns.to_list()
classes = 15
column_details = []
for column_name in columns:
    min_val = min(df[column_name].to_list())
    max_val = max(df[column_name].to_list())
    class_size = round((max_val-min_val)/classes , 2)

    low_bounds = list(np.arange(min_val,max_val,class_size))
    for i in range(len(low_bounds)):
        low_bounds[i] = round(low_bounds[i],2)
    x = low_bounds[-1]
    low_bounds.append(round(x + class_size,2))
    # print(low_bounds)
    # mid_class = round((class_size/2) , 2)
    dict = {}
    # start_mid = mid_class
    # df_temp = df[column_name]
    # print(df_temp)
    # print(len(diabetes_yes) , len(diabetes_no))

    for i in range(classes):
#         # print(start_mid)
        low_bound = low_bounds[i]
        upp_bound = low_bounds[i+1]

        yes_count = diabetes_yes.where(diabetes_yes[column_name]>=low_bound).dropna()
        if i<classes-1:
            yes_count = yes_count.where(yes_count[column_name]<upp_bound).dropna()
        else:
            yes_count = yes_count.where(yes_count[column_name]<=upp_bound).dropna()

        no_count = diabetes_no.where(diabetes_no[column_name]>=low_bound).dropna()
        if i<classes-1:
            no_count = no_count.where(no_count[column_name]<upp_bound).dropna()
        else:
            no_count = no_count.where(no_count[column_name]<=upp_bound).dropna()


#         # print(len(yes_count)/len(diabetes_yes))
#         # print(len(yes_count)/len(diabetes_yes) , len(no_count)/len(diabetes_no))

        dict[low_bounds[i]] = [len(yes_count)/len(diabetes_yes) , len(no_count)/len(diabetes_no)]
#         # print([low_bound,upp_bound])
#         start = round((start_mid + class_size),2)

    column_details.append([column_name, dict , class_size])
print(column_details)

[['Glucose', {0.0: [0.0, 0.007614213197969543], 13.27: [0.0, 0.0], 26.54: [0.0, 0.0], 39.81: [0.0, 0.0], 53.08: [0.0, 0.015228426395939087], 66.35: [0.004545454545454545, 0.0583756345177665], 79.62: [0.02727272727272727, 0.1751269035532995], 92.89: [0.08181818181818182, 0.24111675126903553], 106.16: [0.1409090909090909, 0.17766497461928935], 119.43: [0.1590909090909091, 0.15228426395939088], 132.7: [0.1409090909090909, 0.08121827411167512], 145.97: [0.11818181818181818, 0.0532994923857868], 159.24: [0.11363636363636363, 0.015228426395939087], 172.51: [0.11818181818181818, 0.012690355329949238], 185.78: [0.09545454545454546, 0.01015228426395939]}, 13.27], ['BloodPressure', {0.0: [0.05909090909090909, 0.03553299492385787], 7.6: [0.0, 0.0], 15.2: [0.0, 0.0], 22.8: [0.004545454545454545, 0.0025380710659898475], 30.4: [0.0, 0.0], 38.0: [0.004545454545454545, 0.012690355329949238], 45.6: [0.02727272727272727, 0.050761421319796954], 53.2: [0.05, 0.1446700507614213], 60.8: [0.15454545454545454

In [19]:
def get_outcome(input_data):
    probabilities = []

    for i in range(6):
        value = input_data[i]
        prob_dict = column_details[i][1]
        class_range = column_details[i][2]

        key_list = list(prob_dict.keys())
        
        for key in key_list:
            low = key
            up = key+class_range

            if key_list[-1] == key:
                if value>=low and value <= up:
                    probabilities.append(prob_dict[key])
                    break
            else:
                if value>=low and value < up:
                    probabilities.append(prob_dict[key])
                    break

    yes=1
    no=1
    for i in probabilities:
        if i[0] == 0 and i[1]==0:
            continue
        yes=yes*i[0]
        no=no*i[1]
    
    if yes==1 or no==1:
        return 0

    product = [yes*diabetes_yes_prob , no*diabetes_no_prob]
    total = yes*diabetes_yes_prob + no*diabetes_no_prob

    
    outcome = [product[0]/total, product[1]/total]

    if outcome[0] > outcome[1]:
        return 1
    else:
        return 0

In [20]:
test = x_test.iloc[:,:6].copy()
test_set = []
for i in range(154):
    test_set.append(test.iloc[i].to_list())

by_nb = []
for i in test_set:
    by_nb.append(get_outcome(i))


In [21]:
by_dataset = y_test.to_list()
correct = 0
for i in range(len(by_nb)):
    if by_nb[i]==by_dataset[i]:
        correct = correct + 1

print(correct/len(by_nb)*100)

70.77922077922078


In [22]:
x_test

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Age,Outcome
580,151,90,46,0,42.1,21,1
470,144,82,40,0,41.3,28,0
592,132,80,0,0,34.4,44,1
621,92,76,20,0,24.2,28,0
286,155,84,44,545,38.7,34,0
...,...,...,...,...,...,...,...
205,111,72,28,0,23.9,27,0
531,107,76,0,0,45.3,24,0
552,114,88,0,0,27.8,66,0
734,105,75,0,0,23.3,53,0


In [24]:
print("Enter values for the following columns:\n" + ", ".join(columns) + "\n")
user_input  = input()
user_input = user_input.split(' ')
input_data = [float(i) for i in user_input]
print(input_data)
get_outcome(input_data)

Enter values for the following columns:
Glucose, BloodPressure, SkinThickness, Insulin, BMI, Age

[85.0, 95.0, 0.0, 0.0, 22.0, 20.0]


0

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

clf = GaussianNB()

x_tr, x_te, y_tr, y_te = train_test_split(diabetes.iloc[:,:6],diabetes['Outcome'],test_size=0.2)

clf.fit(x_tr,y_tr)
result = clf.predict(x_te)
accuracy_score(y_te, result) *100

75.32467532467533